In [12]:
import cv2
import ffmpeg
import os
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from PIL import Image
import glob
from data_loader_jorge import RescaleT
from data_loader_jorge import CenterCrop
from data_loader_jorge import ToTensor
from data_loader_jorge import ToTensorLab
from data_loader_jorge import SalObjDatasetFromMem

from model import BASNet

#functions area 

def normPRED(d):
	ma = torch.max(d)
	mi = torch.min(d)

	dn = (d-mi)/(ma-mi)

	return dn

def save_output(image_name,pred,d_dir):

	predict = pred
	predict = predict.squeeze()
	predict_np = predict.cpu().data.numpy()

	im = Image.fromarray(predict_np*255).convert('RGB')
	img_name = image_name.split("/")[-1]
	image = io.imread(image_name)
	imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

	pb_np = np.array(imo)

	aaa = img_name.split(".")
	bbb = aaa[0:-1]
	imidx = bbb[0]
	for i in range(1,len(bbb)):
		imidx = imidx + "." + bbb[i]

	imo.save(d_dir+imidx+'.png')
    
def enmask(in_array):
    
    out_array= np.zeros(shape=(in_array.shape[0],in_array.shape[1]), dtype=bool)
    return 1


def transform_pred(image,pred,d_dir):

	predict = pred
	predict = predict.squeeze()
	predict_np = predict.cpu().data.numpy()
	predict_np2 = np.round(predict_np,0)
    


	im = Image.fromarray(predict_np*255).convert('RGB')
#	img_name = image_name.split("/")[-1]
#	image = io.imread(image_name)
	imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

	pb_np = np.array(imo)
#	pb_np2=enmask(pb_np)
	print(pb_np.shape)

#	aaa = img_name.split(".")
#	bbb = aaa[0:-1]
#	imidx = bbb[0]
#	for i in range(1,len(bbb)):
#		imidx = imidx + "." + bbb[i]

#	f_name=d_dir+imidx+'.jpg'
#	imo.save(f_name)
   
	return pb_np
    
def process_image(img):
    
   test_salobj_dataset = SalObjDatasetFromMem(img_in = img, lbl = [],transform=transforms.Compose([RescaleT(256),ToTensorLab(flag=0)]))
   test_salobj_dataloader = DataLoader(test_salobj_dataset, batch_size=1,shuffle=False,num_workers=1)
   
   for i_test, data_test in enumerate(test_salobj_dataloader):

      inputs_test = data_test['image']
      inputs_test = inputs_test.type(torch.FloatTensor)

      if torch.cuda.is_available():
          inputs_test = Variable(inputs_test.cuda())
      else:
          inputs_test = Variable(inputs_test)

      d1,d2,d3,d4,d5,d6,d7,d8 = net(inputs_test)

      print(d1.shape)
   # normalization
      pred = d1[:,0,:,:]
      pred = normPRED(pred)
   # save results to test_results folder
      img_ret=transform_pred(img,pred,prediction_dir)
      del d1,d2,d3,d4,d5,d6,d7,d8
   return img_ret


def generate_thumbnail(in_filename, out_filename, time, width):
    try:
        (
            ffmpeg
            .input(in_filename, ss=time)
            .filter('scale', width, -1)
            .output(out_filename, vframes=1)
            .overwrite_output()
            .run(capture_stdout=True, capture_stderr=True)
        )
    except ffmpeg.Error as e:
        print(e.stderr.decode(), file=sys.stderr)
        sys.exit(1)
        
#main

#imgtest=cv2.imread("./prediction/image2.jpg")
#cv2.imshow("prueba",imgtest)
#cv2.waitKey(0)
#1 definitions
print("hola")
model_dir = './saved_models/basnet_bsi/basnet.pth'
input_file= "/home/jruano/devs/videos/violin5.mp4"
input_file2= "/home/jruano/devs/videos/guitar2.mp4"

bkg_file="/home/jruano/Pictures/sunset.jpg"
output_video="video_comp.mp4"
video_prueba="video_test2.mp4"
tmp_dir="./tmp/"
prediction_dir="./prediction/"

firsttime=0
frame_number=0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

height = 960
width = 540



# --------- 3. model define ---------
print("...load BASNet...")
net = BASNet(3,1)
net.load_state_dict(torch.load(model_dir))
if torch.cuda.is_available():
	net.cuda()
net.eval()

#4 body



bkg=cv2.imread(bkg_file)

videowriter = cv2.VideoWriter(output_video, fourcc, 20.0, (width*2,height))
videowriter2 = cv2.VideoWriter(video_prueba, fourcc, 20.0, (width,height))

cap = cv2.VideoCapture(input_file); 
cap2 = cv2.VideoCapture(input_file2); 


while(1): 
# read frames 
   print("Frame Number is",frame_number);
   frame_number=frame_number+1
   ret2,img = cap.read();
   ret3,img2 = cap2.read();

   print("inferencing image:",frame_number)
   img_ret=process_image(img)
   img_ret2=process_image(img2)



   vis = np.concatenate((img, img2), axis=1)
   vis2 = np.concatenate((img_ret, img_ret2), axis=1)

   final1=cv2.bitwise_and(vis,vis2)
   final15=cv2.bitwise_not(vis2)
   final25=cv2.bitwise_and(bkg,final15)
   final2=cv2.bitwise_or(bkg,vis2)
#      final=final1+final2
   final=cv2.bitwise_or(final1,final25)

#      final=cv2.bitwise_and(img,bkg,mask=img_ret)


#      img_ret=cv2.imread(img_ret_file)
#      cv2.imshow("ejemplo",final1)
#      cv2.waitKey(0)
#      cv2.imshow("ejemplo",final2)
#      cv2.waitKey(0)

   videowriter.write(final)
   videowriter2.write(final1)
#   videowriter.write(img) 
   if frame_number==200 :
            break;

 

cap.release(); 
videowriter.release()
videowriter2.release()

  

cv2.destroyAllWindows(); 



#generate_thumbnail("/home/jruano/devs/videos/video1.mp4","fichero.jpg",1,200)


hola
...load BASNet...


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f37a6a0dd90>
Traceback (most recent call last):
  File "/home/jruano/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 962, in __del__
    self._shutdown_workers()
  File "/home/jruano/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 942, in _shutdown_workers
    w.join()
  File "/home/jruano/anaconda3/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/jruano/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/home/jruano/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


Frame Number is 0
inferencing image: 1
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 1
inferencing image: 2
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 2
inferencing image: 3
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 3
inferencing image: 4
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 4
inferencing image: 5
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 5
inferencing image: 6
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 6
inferencing image: 7
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 7
inferencing image: 8
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)


torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 65
inferencing image: 66
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 66
inferencing image: 67
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 67
inferencing image: 68
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 68
inferencing image: 69
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 69
inferencing image: 70
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 70
inferencing image: 71
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 71
inferencing image: 72
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 72
inferencing image: 73
torch.Size([1, 1, 256, 256]

torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 130
inferencing image: 131
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 131
inferencing image: 132
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 132
inferencing image: 133
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 133
inferencing image: 134
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 134
inferencing image: 135
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 135
inferencing image: 136
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 136
inferencing image: 137
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Numbe

torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 194
inferencing image: 195
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 195
inferencing image: 196
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 196
inferencing image: 197
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 197
inferencing image: 198
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 198
inferencing image: 199
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
Frame Number is 199
inferencing image: 200
torch.Size([1, 1, 256, 256])
(960, 540, 3)
torch.Size([1, 1, 256, 256])
(960, 540, 3)
